In [1]:
import numpy as np
import math
from collections import namedtuple

In [2]:
Direction = namedtuple('Direction', ('D0','D90','D180','D270'))

# 共同住宅の躯体性能の簡易評価法

## I. 入力項目

<center>住棟全体に関する入力項目</center>

| No. | 入力項目 | 単位 | 記号 | 備考 |
| --- | --- | --- | --- | --- |
| G1 | 地域の区分 | - | - | 1～8地域 |
| G2 | 建物の構造 | - | - | RC, S, W |
| G3 | 建物の階数 | 階 | $N$ | - |
| G4 | 建物全体の高さ | m | $H_{total}$ | - |
| G5 | 主開口部の方位 | - | - | 8方位 |
| G6 | 各階の床面積の合計 | m<sup>2</sup> | $A_{f,total}$ | 階ごとに入力する |
| G7 | 各階の住戸数 | 戸 | $n$ | 階ごとに入力する |

In [3]:
d_General = {
    "Region"               : 6,                        # 地域の区分
    "Structure"            : "RC",                     # 建物の構造
    "Levels_Number"        : 3,                        # 建物の階数
    "Height"               : 9.1,                      # 建物の高さ(m)
    "OpeningMainDirection" : "south",                  # 主開口部の方位
    "FloorArea"            : [231.46, 222.34, 222.34], # 各階の床面積の合計(m2)
    "HouseNumber"          : [4,4,4]                   # 各階の住戸数
}

<center>専有部に関する入力項目</center>

| No. | 入力項目 | 単位 | 記号 | 備考 |
| --- | --- | --- | --- | --- |
| P1 | 各階の外周長 外気と接する部分 | m | | 階ごとに入力する |
| P2 | 各階の外周長 空調された共用部や非住宅部と接する部分 | m | | 階ごとに入力する |
| P3 | 各階の外周長 住戸と接する部分 | m | | 階ごとに入力する (住棟を複数の方位に分けて計算する場合のみ入力) |
| P4 | 各階の窓面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| P5 | 各階(最上階を除く)の屋根面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| P6 | 各階(最下階を除く)の外気床の面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| P7 | 外壁の仕様 熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P8 | 外壁の仕様 断熱位置 | - | | inside=内断熱, outside=外断熱, non=無断熱 から選択 |
| P9 | 外壁の仕様 断熱補強の仕様 | - | | spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択 |
| P10 | 屋根の仕様 熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P11 | 屋根の仕様 断熱位置 | - | | inside=内断熱, outside=外断熱, non=無断熱 から選択 |
| P12 | 屋根の仕様 断熱補強の仕様 | - | | spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択 |
| P13 | 最下階床の仕様 熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P14 | 最下階床の仕様 断熱位置 | - | | inside=内断熱, outside=外断熱, non=無断熱 から選択 |
| P15 | 最下階床の仕様 断熱補強の仕様 | - | | spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択 |
| P16 | 界壁の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P17 | 界床の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P18 | 窓の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P19 | 窓の日射熱取得率(η値) | - | | |

In [4]:
d_PrivateArea = {
    "PeriLengthOutside"   : [66.56, 66.56, 66.56], # 各階の外周長 外気と接する部分(m)
    "PeriLengthInside"    : [0.0, 0.0, 0.0],       # 各階の外周長 空調された共用部や非住宅部と接する部分(m)
    "PeriLengthPrivate"   : [0.0, 0.0, 0.0],       # 各階の外周長 住戸と接する部分(住棟を複数の方位に分けて計算する場合のみ)(m)
    "WindowArea"          : [30.48, 30.64, 30.64], # 各階の窓面積の合計(m2)
    "RoofArea"            : [0.0, 0.0, None],      # 各階(最上階を除く)の屋根面積の合計(m2)
    "OutsideFloorArea"    : [None, 0.0, 0.0],      # 各階(最下階を除く)の外気床の面積の合計(m2)
    "Wall"                : {                      # 外壁の仕様
        "UValue" : 0.785,    # 熱貫流率(U値)(W/m2K)
        "InsPos" : "inside", # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
        "HBSpec" : "spec1"   # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
    },
    "Roof"                : {                      # 屋根の仕様
        "UValue" : 0.417,     # 熱貫流率(U値)(W/m2K)
        "InsPos" : "outside", # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
        "HBSpec" : "spec1"    # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
    },
    "Floor"               : {                      # 最下階床の仕様
        "UValue" : 0.507,     # 熱貫流率(U値)(W/m2K)
        "InsPos" : "outside", # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
        "HBSpec" : "null"     # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
    },
    "InsideWallUValue"    : 2.34,                  # 界壁の熱貫流率(U値)(W/m2K)
    "InsideFloorUValue"   : 1.796,                 # 界床の熱貫流率(U値)(W/m2K)
    "WallUValue"          : 4.65,                  # 窓の熱貫流率(U値)(W/m2K)
    "WallEtaValue"        : 0.79                   # 窓の日射熱取得率(η値)    
}

<center>共用部に関する入力項目</center>

| No. | 入力項目 | 単位 | 記号 | 備考 |
| --- | --- | --- | --- | --- |
| C1 | 各階の共用部・非住宅部の床面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| C2 | 各階(最上階を除く)の屋根面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| C3 | 各階(最下階を除く)の外気床の面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| C4 | 外壁の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| C5 | 界床の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| C6 | 外気床の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| C7 | 窓の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| C8 | 窓の日射熱取得率(η値) | - | |  |

In [5]:
d_PublicArea = {
    "FloorArea"          : [68.82, 59.7, 59.7], # 各階の共用部・非住宅部の床面積の合計(m2)
    "RoofArea"           : [9.125, 0.0, None],  # 各階(最上階を除く)の屋根面積の合計(m2)
    "OutsideFloorArea"   : [None, 0.0, 0.0],    # 各階(最下階を除く)の外気床の面積の合計(m2)
    "WallUValue"         : 0.00,                # 外壁の熱貫流率(U値)(W/m2K)
    "InsideFloorUValue"  : 0.00,                # 界床の熱貫流率(U値)(W/m2K)
    "OutsideFloorUValue" : 0.00,                # 外気床の熱貫流率(U値)(W/m2K)
    "WindowUValue"       : 0.00,                # 窓の熱貫流率(U値)(W/m2K)
    "WindowEtaValue"     : 0.00                 # 窓の日射熱取得率(η値)
}

In [6]:
d = {
    "General"     : d_General,
    "PrivateArea" : d_PrivateArea,
    "PublicArea"  : d_PublicArea
}

## II. 計算方法

### 1. 各階全体の面積等

#### 1) 各階の階高

各階の階高$H$(m)は次式により表される。

$$
H = \frac{H_{total}}{N}
$$

$H_{total}$: 建物全体の高さ(m)  
$N$: 建物の階数  

In [7]:
def get_H(H_total, N):
    return [ H_total / N ] * N

#### 計算例

建物全体の高さ 9.1 (m), 建物の階数 3階

In [8]:
get_H(9.1,3)

[3.033333333333333, 3.033333333333333, 3.033333333333333]

#### 2) 専有部の床面積合計(各階)

各階の専有部(住戸部）の床面積の合計 $A_{f,h,total}$は次式により表される。  

$$ A_{f,h,total} = A_{f,total} - A_{f,c,total} $$

$ A_{f,total} $: 各階の床面積の合計(m<sup>2</sup>)  
$ A_{f,c,total} $: 各階の共用部、非住宅部の床面積の合計(m<sup>2</sup>)

In [9]:
def get_A_f_h_total(A_f_total, A_f_c_total):
    return [ x - y for (x,y) in zip(A_f_total, A_f_c_total) ]

#### 計算例

各階の床面積の合計(m<sup>2</sup>)  
1階:231.46,2階:222.34,3階:222.34  

各階の共用部、非住宅部の床面積の合計(m<sup>2</sup>)  
1階:68.82,2階:59.7,3階:59.7

In [10]:
get_A_f_h_total( [231.46, 222.34, 222.34], [68.82,59.7,59.7] )

[162.64000000000001, 162.64, 162.64]

#### 3) 専有部（住戸部）全体の長辺および短辺の長さ(各階)

各階における専有部(住戸部)全体の平面形状は矩形を想定し、その長辺が主方位(主方位+180°)側にあるもとの仮定し、長辺と短辺の長さは次式を連立して求めることとする。

$$ l_{1,total} \ge l_{2,total} $$
$$ l_{1,total} \times l_{2,total} = A_{f,h,total} $$
$$ 2 \times ( l_{1,total} + l_{2,total} ) = L $$

これらを連立すると、

$$ l_{1,total} = \frac{ \frac{L}{2} + \sqrt{ (\frac{L}{2})^2 - 4 \times A_{f,h,total} } }{2} $$

$$ l_{2,total} = \frac{ \frac{L}{2} - \sqrt{ (\frac{L}{2})^2 - 4 \times A_{f,h,total} } }{2} $$

となる。

$ A_{f,h,total} $: 各階における専有部(住戸部)の床面積の合計(m<sup>2</sup>)  
$ l_{1,total} $: 各階における専有部(住戸部)全体の長辺の長さ(m)  
$ l_{2,total} $: 各階における専有部(住戸部)全体の短辺の長さ(m)  
$ L_i$: 各階における専有部(住戸部)全体の外皮の周長(m)

In [11]:
def get_l_1_total(L, A_f_h_total):
    return [ ( (l/2) + math.sqrt( (l/2)**2 - 4 * a ) )/2 for (l,a) in zip(L, A_f_h_total) ]

In [12]:
def get_l_2_total(L, A_f_h_total):
    return [ ( (l/2) - math.sqrt( (l/2)**2 - 4 * a ) )/2 for (l,a) in zip(L, A_f_h_total) ]

#### 計算例

専有部(住戸部)の床面積の合計(m<sup>2</sup>)  
1階:66.56, 2階:66.56, 3階:66.56

専有部(住戸部)全体の外皮の周長(m)  
1階:162.6, 2階:162.6, 3階:162.6

In [13]:
get_l_1_total( [66.56,66.56,66.56], [162.6,162.6,162.6] )

[27.330631412596734, 27.330631412596734, 27.330631412596734]

In [14]:
get_l_2_total([ 66.56,66.56,66.56], [162.6,162.6,162.6] )

[5.949368587403267, 5.949368587403267, 5.949368587403267]

#### 4) 専有部(住戸部)の屋根面積の合計(各階)

各階の専有部(住戸部)の屋根面積の合計$A_{r,h,total}$ (m<sup>2</sup>)は次式により表される。

$$
A_{r,h,total} = \left\{
    \begin{array}{ll}
        A_{r,h,total,i,input} & (i \ne N) \\
        A_{f,h,total,i} & (i = N)
    \end{array}
\right.
$$

$i$階における専有部（住戸部）の屋根面積の合計は最上階とそれ以外の階に分けて求める。最上階以外は入力値とし、最上階は床面積の合計に等しいものとする。なお、最上階以外に屋根がある場合は、その階の主方位側かつ各住戸に均等に位置するものとする。

$N$: 建物の階数(階)  
$A_{r,h,total,i}$: $i$階における専有部(住戸部)の屋根面積の合計(m<sup>2</sup>)

In [15]:
def get_A_r_h_total(A_r_h_total_input, A_f_h_total):
    
    N = len(A_f_h_total) # 階数
    
    return [
        A_r if idx != N-1 else A_f     # 最上階以外はA_r(屋根)の値、最上階のみA_f(床)の値 (最上階は床面積=屋根面積が自明なため)
        for idx,(A_r,A_f) in enumerate(zip(A_r_h_total_input, A_f_h_total))
    ]

#### 計算例

各階(最上階を除く)の屋根面積の合計(m<sup>2</sup>)  
1階:0.0, 2階:0.0, 3階:非入力項目

専有部(住戸部)の床面積の合計(m<sup>2</sup>)  
1階:162.6, 2階:162.6, 3階:162.6

In [16]:
get_A_r_h_total( [0.0, 0.0, None], [162.6, 162.6, 162.6] )

[0.0, 0.0, 162.6]

#### 5) 専有部(住戸部)の外気に接する床の面積の合計

$i$階における専有部(住戸部)の外気に接する床の面積の合計$A_{of,h,total,i}$ (m<sup>2</sup>) は次式により表される。  

$$
A_{of,h,total,i} = \left\{
    \begin{array}{ll}
        0 & (i = 1) \\
        A_{of,h,total,i,input} & (i \ne 1)
    \end{array}
\right.
$$

$i$階における専有部(住戸部)の外気に接する床の面積の合計は、1階(最下階)は0とし、1階以外は入力値とする。  
なお、最下階以外に外気に接する床がある場合は、その階の主方位側かつ各住戸に均等に位置するものとする。

$A_{of,h,total,i}$: $i$階における専有部(住戸部)の外気に接する床の面積の合計(m<sup>2</sup>)

In [17]:
def get_A_of_h_total(A_of_h_total_input):
    
    return [
        0.0 if idx == 0 else A_of     # 1階部分は0m<sup>2</sup>とする。
        for idx, A_of in enumerate(A_of_h_total_input)
    ]

#### 計算例

各階(最下階を除く)の外気床の面積の合計(m<sup>2</sup>)  
1階:非入力項目, 2階:5.0, 3階:5.0

In [18]:
get_A_of_h_total([None,5.0,5.0])

[0.0, 5.0, 5.0]

#### 6) 窓面積の合計

各階における方位$j$の窓面積の合計は次式により算出する。なお、主方位を$D0$とし、方位の角度は上から見て時計回りに増加するものとし、$D90$で主方位+90度、$D180$で主方位+180度、$D270$で主方位+270度とする。（以下、同じ）。

$$
A_{win,h,total,D0} = A_{win,h,total} \times ( 1 - \frac{0.24}{n} ) \times 0.64
$$

$$
A_{win,h,total,D90} = A_{win,h,total} \times \frac{0.12}{n}
$$

$$
A_{win,h,total,D180} = A_{win,h,total} \times ( 1 - \frac{0.24}{n} ) \times 0.36
$$

$$
A_{win,h,total,D270} = A_{win,h,total} \times \frac{0.12}{n}
$$

$A_{win,h,total}$: 各階における窓面積の合計(m<sup>2</sup>)  
$A_{win,h,total,j}$: 各階における方位$j$の窓面積の合計(m<sup>2</sup>)  
$n$: 各階における住戸数(戸)

In [19]:
def get_A_win_h_total(A_win_h_total,n):
    def f(A,n_i):
        return Direction( A * (1-0.24/n_i) * 0.64, A * 0.12/n_i, A * (1-0.24/n_i) * 0.36, A * 0.12/n_i )

    return [ f(A,n_i) for (A,n_i) in zip(A_win_h_total,n)]

#### 計算例

各階の窓面積の合計(m<sup>2</sup>)  
1階:30.48, 2階:30.64, 3階:30.64

各階の住戸数
1階:4戸, 2階:4戸, 3階:4戸

In [20]:
get_A_win_h_total([30.48,30.64,30.64],[4,4,4])

[Direction(D0=18.336768, D90=0.9144, D180=10.314432, D270=0.9144),
 Direction(D0=18.433024, D90=0.9192, D180=10.368576, D270=0.9192),
 Direction(D0=18.433024, D90=0.9192, D180=10.368576, D270=0.9192)]

### 2. 各階における各住戸の面積等

#### 1) 屋根の面積

各階における各住戸の屋根面積$A_{r,h}$(m<sup>2</sup>)は次式により算出する。

$$
A_{r,h} = \frac{A_{r,h,total}}{n}
$$

$A_{r,h}$: 各階における各住戸の屋根面積 (m<sup>2</sup>)  
$A_{r,h,total}$: 各階における専有部(住戸部)全体の屋根面積の合計 (m<sup>2</sup>)  
$n$: 各階における住戸数 (戸)

In [21]:
def get_A_r_h(A_r_h_total, n):
    return [ [A_r_h_total_i / n_i] * n_i for (A_r_h_total_i, n_i) in zip(A_r_h_total, n) ]

#### 計算例

専有部(住戸部)の屋根面積の合計(m<sup>2</sup>)
1階:0.0, 2階:0.0, 3階:162.6

各階の住戸数  
1階:4, 2階:4, 3階:4, 

In [22]:
get_A_r_h([0.0,0.0,162.6],[4,4,4])

[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [40.65, 40.65, 40.65, 40.65]]

#### 2) 窓の面積

各階の各方位における各住戸の窓面積$A_{win,h,D0,k}$, $A_{win,h,D90,k}$, $A_{win,h,D180,k}$, $A_{win,h,D270,k}$は、住戸位置ごとに次式により算出する。

$$
A_{win,h,D0,k} = \frac{ A_{win,h,total,D0} } { n }
$$

$$
A_{win,h,D90,k} = \left\{
    \begin{array}{ll}
        A_{win,h,total,D90} & (k=1) \\
        0 & (k \ne 1)
    \end{array}
\right.
$$

$$
A_{win,h,D180,k} = \frac{ A_{win,h,total,D180} } { n }
$$

$$
A_{win,h,D270,k} = \left\{
    \begin{array}{ll}
        A_{win,h,total,D270} & (k=n) \\
        0 & (k \ne n)
    \end{array}
\right.
$$

$A_{win,h,D0,k}$: 各階における住戸kの主方位の窓面積(m<sup>2</sup>)  
$A_{win,h,D90,k}$: 各階における住戸kの主方位+90°の窓面積(m<sup>2</sup>)  
$A_{win,h,D180,k}$: 各階における住戸kの主方位+180°の窓面積(m<sup>2</sup>)  
$A_{win,h,D270,k}$: 各階における住戸kの主方位+270°の窓面積(m<sup>2</sup>)  
$A_{win,h,total,D0}$: 各階における主方位の窓面積の合計(m<sup>2</sup>)  
$A_{win,h,total,D90}$: 各階における主方位+90°の窓面積の合計(m<sup>2</sup>)  
$A_{win,h,total,D180}$: 各階における主方位+180°の窓面積の合計(m<sup>2</sup>)  
$A_{win,h,total,D270}$: 各階における主方位+270°の窓面積の合計(m<sup>2</sup>)  

In [23]:
def get_A_win_h(A_win_h_total, n):
    def f(A_win_h_total_i, n_i):
        return [ Direction ( A_win_h_total_i[0] / n_i,
                             A_win_h_total_i[1] if k == 0 else 0,
                             A_win_h_total_i[2] / n_i,
                             A_win_h_total_i[3] if k == n_i-1 else 0
                           ) for k in range(n_i) ]
    return [ f(A_win_h_total_i, n_i) for (A_win_h_total_i, n_i) in zip(A_win_h_total, n) ]

#### 計算例

各階における窓面積の合計(m<sup>2</sup>)  
1階: 主方位+0°:18.336768, 主方位+90°: 0.9144, 主方位+180°: 10.314432, 主方位+270°: 0.9144  
2階: 主方位+0°:18.433024, 主方位+90°: 0.9192, 主方位+180°: 10.368576, 主方位+270°: 0.9192
3階: 主方位+0°:18.433024, 主方位+90°: 0.9192, 主方位+180°: 10.368576, 主方位+270°: 0.9192

各階における住戸数
1階:4, 2階:4, 3階:4

In [24]:
get_A_win_h([[18.336768, 0.9144, 10.314432, 0.9144], [18.433024, 0.9192, 10.368576, 0.9192], [18.433024, 0.9192, 10.368576, 0.9192]],[4,4,4])

[[Direction(D0=4.584192, D90=0.9144, D180=2.578608, D270=0),
  Direction(D0=4.584192, D90=0, D180=2.578608, D270=0),
  Direction(D0=4.584192, D90=0, D180=2.578608, D270=0),
  Direction(D0=4.584192, D90=0, D180=2.578608, D270=0.9144)],
 [Direction(D0=4.608256, D90=0.9192, D180=2.592144, D270=0),
  Direction(D0=4.608256, D90=0, D180=2.592144, D270=0),
  Direction(D0=4.608256, D90=0, D180=2.592144, D270=0),
  Direction(D0=4.608256, D90=0, D180=2.592144, D270=0.9192)],
 [Direction(D0=4.608256, D90=0.9192, D180=2.592144, D270=0),
  Direction(D0=4.608256, D90=0, D180=2.592144, D270=0),
  Direction(D0=4.608256, D90=0, D180=2.592144, D270=0),
  Direction(D0=4.608256, D90=0, D180=2.592144, D270=0.9192)]]

### 3) 玄関ドアの面積

各階の各住戸の各方位における玄関ドアの面積$A_{door,h,D0}$, $A_{door,h,D90}$, $A_{door,h,D180}$, $A_{door,h,D270}$ (m<sup>2</sup>)は次式により算出する。

$$
A_{door,h,D0} = 0
$$

$$
A_{door,h,D90} = 0
$$

$$
A_{door,h,D180} = 1.6
$$

$$
A_{door,h,D270} = 0
$$

$A_{door,h}$: 各階における各住戸の玄関ドアの面積(m<sup>2</sup>)

In [25]:
def get_A_door_h(n):
    return [ [ Direction ( 0.0,0.0,1.6,0.0) for k in range(n_i) ] for n_i in n ]

#### 計算例

各階の住戸数  
1階:4, 2階:4, 3階:4

In [26]:
get_A_door_h([4,4,4])

[[Direction(D0=0.0, D90=0.0, D180=1.6, D270=0.0),
  Direction(D0=0.0, D90=0.0, D180=1.6, D270=0.0),
  Direction(D0=0.0, D90=0.0, D180=1.6, D270=0.0),
  Direction(D0=0.0, D90=0.0, D180=1.6, D270=0.0)],
 [Direction(D0=0.0, D90=0.0, D180=1.6, D270=0.0),
  Direction(D0=0.0, D90=0.0, D180=1.6, D270=0.0),
  Direction(D0=0.0, D90=0.0, D180=1.6, D270=0.0),
  Direction(D0=0.0, D90=0.0, D180=1.6, D270=0.0)],
 [Direction(D0=0.0, D90=0.0, D180=1.6, D270=0.0),
  Direction(D0=0.0, D90=0.0, D180=1.6, D270=0.0),
  Direction(D0=0.0, D90=0.0, D180=1.6, D270=0.0),
  Direction(D0=0.0, D90=0.0, D180=1.6, D270=0.0)]]

### 4) 外壁の面積

各階の住戸$k$の各方位における外壁面積$A_{ow,h,k,D0}$, $A_{ow,h,k,D90}$, $A_{ow,h,k,D180}$, $A_{ow,h,k,D270}$は、住戸位置ごとに次式により算出する。なお、主方位+180°に玄関前を想定した窪み(幅1m、奥行き1m)を想定する。

$$
A_{ow,h,k,D0} = \frac{ l_{1,total} }{ n } \times H -  A_{win,h,D0} - A_{door,h,D0}
$$

$$
A_{ow,h,k,D90} = \left\{
    \begin{array}{ll}
        ( l_{2,total} + 1 ) \times H - A_{win,h,k,D90} - A_{door,h,D90} & (k=1) \\
        1 \times H - A_{win,h,k,D90} - A_{door,h,D90} & (k \ne 1)
    \end{array}
\right.
$$

$$
A_{ow,h,k,D180} = \frac{ l_{1,total} }{ n } \times H -  A_{win,h,D180} - A_{door,h,D180}
$$

$$
A_{ow,h,k,D270} = \left\{
    \begin{array}{ll}
        ( l_{2,total} + 1 ) \times H - A_{win,h,k,D270} - A_{door,h,D270} & (k=n) \\
        1 \times H - A_{win,h,k,D270} - A_{door,h,D270} & (k \ne n)
    \end{array}
\right.
$$

$A_{ow,h,k,D0}$: 各階の各住戸における主方位の外壁面積 (m<sup>2</sup>)  
$A_{ow,h,k,D90}$: 各階の各住戸における主方位+90°の外壁面積 (m<sup>2</sup>)  
$A_{ow,h,k,D180}$: 各階の各住戸における主方位+180°の外壁面積 (m<sup>2</sup>)  
$A_{ow,h,k,D270}$: 各階の各住戸における主方位+270°の外壁面積 (m<sup>2</sup>)  
$l_{1,total}$: 各階における専有部(住戸部)全体の長辺の長さ (m)  
$l_{2,total}$: 各階における専有部(住戸部)全体の短辺の長さ (m)  
$n$: 各階における住戸数 (戸)  
$H$: 各階の階高 (m)  
$A_{win,h,D0}$: 各階の各住戸における主方位の窓面積 (m<sup>2</sup>)  
$A_{win,h,i,D90}$: 各階の各住戸における主方位+90°の窓面積 (m<sup>2</sup>)  
$A_{win,h,i,D180}$: 各階の各住戸における主方位+180°の窓面積 (m<sup>2</sup>)  
$A_{win,h,i,D270}$: 各階の各住戸における主方位+270°の窓面積 (m<sup>2</sup>)  
$A_{door,h,i,D180}$: 各階の各住戸における玄関ドアの面積 (m<sup>2</sup>)

In [27]:
def get_A_ow_h(l_1_total,l_2_total,H,n,A_win_h,A_door_h):
    def f( l_1_total_i, l_2_total_i, H_i,n_i, A_win_h_i, A_door_h_i ):
        return [ Direction ( l_1_total_i / n_i * H_i - A_win_h_i[k].D0 - A_door_h_i[k].D0,
                             ( ( l_2_total_i + 1 ) * H_i if k == 0 else 1 * H_i ) -  A_win_h_i[k].D90 - A_door_h_i[k].D90,
                             l_1_total_i / n_i * H_i  - A_win_h_i[k].D180 - A_door_h_i[k].D180,
                             ( ( l_2_total_i + 1 ) * H_i if k == n_i-1 else 1 * H_i ) - A_win_h_i[k].D270 - A_door_h_i[k].D270
                           ) for k in range(n_i) ]
    return [ f( l_1_total_i, l_2_total_i, H_i, n_i, A_win_h_i, A_door_h_i )
            for ( l_1_total_i, l_2_total_i, H_i, n_i, A_win_h_i, A_door_h_i )
            in zip(l_1_total, l_2_total, H, n, A_win_h, A_door_h) ]    

#### 計算例

専有部(住戸部)全体の長辺の長さ 1～3階: 27.33 (m)  
専有部(住戸部)全体の短辺の長さ 1～3階: 5.95 (m)  
各階の住戸数 1～3階: 4住戸  
各階の各住戸の窓面積 (m<sup>2</sup>)  
 主方位+0°  : 4.584192  
 主方位+90° : 主方位+90°壁を有する妻側住戸のみ 0.9144 / それ以外の住戸 0.0  
 主方位+180°: 2.578608  
 主方位+270°: 主方位+90°壁を有する妻側住戸のみ 0.9144 / それ以外の住戸 0.0  
各階の各住戸のドア面積 (m<sup>2</sup>)  
主方位+270°のみ 1.8, それ以外 0.0

In [28]:
get_A_ow_h([27.33,27.33,27.33],[5.95,5.95,5.95],[3.03,3.03,3.03],[4,4,4],
           [ [ Direction(4.584192, 0.9144, 2.578608, 0.0   ),
               Direction(4.584192, 0.0,    2.578608, 0.0   ),
               Direction(4.584192, 0.0,    2.578608, 0.0   ),
               Direction(4.584192, 0.0,    2.578608, 0.9144) ],
             [ Direction(4.584192, 0.9144, 2.578608, 0.0   ),
               Direction(4.584192, 0.0,    2.578608, 0.0   ),
               Direction(4.584192, 0.0,    2.578608, 0.0   ),
               Direction(4.584192, 0.0,    2.578608, 0.9144) ],
             [ Direction(4.584192, 0.9144, 2.578608, 0.0   ),
               Direction(4.584192, 0.0,    2.578608, 0.0   ),
               Direction(4.584192, 0.0,    2.578608, 0.0   ),
               Direction(4.584192, 0.0,    2.578608, 0.9144) ] ],
           [ [ Direction(0.0, 0.0, 1.8, 0.0),
               Direction(0.0, 0.0, 1.8, 0.0),
               Direction(0.0, 0.0, 1.8, 0.0), 
               Direction(0.0, 0.0, 1.8, 0.0) ],
             [ Direction(0.0, 0.0, 1.8, 0.0),
               Direction(0.0, 0.0, 1.8, 0.0),
               Direction(0.0, 0.0, 1.8, 0.0), 
               Direction(0.0, 0.0, 1.8, 0.0) ],
             [ Direction(0.0, 0.0, 1.8, 0.0),
               Direction(0.0, 0.0, 1.8, 0.0),
               Direction(0.0, 0.0, 1.8, 0.0), 
               Direction(0.0, 0.0, 1.8, 0.0) ] ] )

[[Direction(D0=16.118282999999998, D90=20.144099999999998, D180=16.323866999999996, D270=3.03),
  Direction(D0=16.118282999999998, D90=3.03, D180=16.323866999999996, D270=3.03),
  Direction(D0=16.118282999999998, D90=3.03, D180=16.323866999999996, D270=3.03),
  Direction(D0=16.118282999999998, D90=3.03, D180=16.323866999999996, D270=20.144099999999998)],
 [Direction(D0=16.118282999999998, D90=20.144099999999998, D180=16.323866999999996, D270=3.03),
  Direction(D0=16.118282999999998, D90=3.03, D180=16.323866999999996, D270=3.03),
  Direction(D0=16.118282999999998, D90=3.03, D180=16.323866999999996, D270=3.03),
  Direction(D0=16.118282999999998, D90=3.03, D180=16.323866999999996, D270=20.144099999999998)],
 [Direction(D0=16.118282999999998, D90=20.144099999999998, D180=16.323866999999996, D270=3.03),
  Direction(D0=16.118282999999998, D90=3.03, D180=16.323866999999996, D270=3.03),
  Direction(D0=16.118282999999998, D90=3.03, D180=16.323866999999996, D270=3.03),
  Direction(D0=16.11828299

### 5) 外気に接する床の面積

階$i$の各住戸における外気に接する床の面積$A_{of,h,i}$(m<sup>2</sup>)は次式により算出する。

$$
A_{of,h,i} = \left\{
    \begin{array}{ll}
        0 & ( i = 1 ) \\
        \frac{ A_{of,h,total,i} }{ n_i } & ( i \ge 2 )
    \end{array}
\right.
$$

$A_{of,h,i}$: 階$i$の各住戸における外気に接する床の面積 (m<sup>2</sup>)  
$A_{of,h,total,i}$: 階$i$における専有部（住戸部）の外気に接する床の面積の合計 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数 (戸)

In [29]:
def get_A_of_h(A_of_h_total,n):
    return [ [ A_of_h_total_i / n_i ] * n_i for ( A_of_h_total_i, n_i ) in zip( A_of_h_total, n ) ]

#### 計算例

専有部(住戸部)の外気床面積の合計 1階:0.0, 2階:2.0, 3階:2.0 (m<sup>2</sup>)  
各階の住戸数: 1～4階: 4 住戸

In [30]:
get_A_of_h([0.0,2.0,2.0],[4,4,4])

[[0.0, 0.0, 0.0, 0.0], [0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5]]

### 6) 界壁の面積

各階の住戸$k$の界壁面積$A_{iw,h,k}$(m<sup>2</sup>)は次式により算出する。

$ n = 1 $ (各階に1住戸しかない場合)の場合、

$$
A_{iw,h,k} = 0.0
$$

$ n > 1 $ の場合、

$$
A_{iw,h,k} = \left\{
    \begin{array}{ll}
        l_{2,total} \times H & ( k = 1 ) \\
        l_{2,total} \times H \times 2 & ( (k \ne 1) \& (k \ne n) ) \\
        l_{2,total} \times H & ( k = n )
    \end{array}
\right.
$$

$n$: 各階における住戸数(戸)  
$A_{iw,h,k}$: 各階の住戸$k$における界壁の面積 (m<sup>2</sup>)  
$l_{2,total}$: 各階における専有部(住戸部)全体の短辺の長さ (m)  
$H$: 各階の階高 (m)

In [31]:
def get_A_iw_h(l_2_total, H, n):
    def f(l_2_total_i, H_i, n_i, k):
        if k == 0:
            return l_2_total_i * H_i
        elif k == n_i - 1:
            return l_2_total_i * H_i
        else:
            return l_2_total_i * H_i * 2
    return [ [0.0] if n_i == 1 else [ f(l_2_total_i, H_i, n_i, k)  for k in range(n_i) ]
             for (l_2_total_i, H_i, n_i) in zip(l_2_total, H, n) ]

#### 計算例

各階における専有部(住戸部)全体の短辺の長さ 6.0 (m)  
各階の階高 3.0 (m)  
各階における住戸数 4 (戸)

In [32]:
get_A_iw_h([6.0,6.0,6.0], [3.0,3.0,3.0], [4,4,4,4])

[[18.0, 36.0, 36.0, 18.0], [18.0, 36.0, 36.0, 18.0], [18.0, 36.0, 36.0, 18.0]]

各階における専有部(住戸部)全体の短辺の長さ 6.0 (m)  
各階の階高 3.0 (m)  
各階における住戸数 1 (戸)

In [33]:
get_A_iw_h([6.0,6.0,6.0], [3.0,3.0,3.0], [1,1,1,1])

[[0.0], [0.0], [0.0]]

### 7) 界床の面積

階$i$の各住戸における界床の面積$A_{if,h,i}$ (m<sup>2</sup>) は、次式により算出する。

$$
A_{if,h,i} = \left\{
    \begin{array}{ll}
        0 & (i=1)\\
        \frac{ A_{f,h,total,i} }{ n_i } - A_{of,h,i} & ( i \ge 2 )
    \end{array}
\right.
$$

$A_{if,h,i}$: 階$i$の各住戸における界床面積 (m<sup>2</sup>)  
$A_{f,h,total,i}$: 階$i$の専有部(住戸部)の床面積の合計 (m<sup>2</sup>)  
$A_{of,h,i}$: 階$i$の各住戸における外気に接する床の面積 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数(戸)

In [34]:
def get_A_if_h(A_f_h_total, A_of_h, n):
    def f(A_f_h_total_i, A_of_h_i, n_i):
        return [ x - y for (x,y) in zip( [ A_f_h_total_i / n_i ] * n_i, A_of_h_i ) ]
    return [ [0.0] * n_i if i == 0 else f(A_f_h_total_i, A_of_h_i, n_i)
             for ( i, (A_f_h_total_i, A_of_h_i, n_i) ) in enumerate(zip(A_f_h_total, A_of_h, n)) ]

#### 計算例

各階の専有部(住戸部)の床面積の合計 1階: 162.6, 2階: 162.6, 3階: 162.6 (m<sup>2</sup>)  
各階の各住戸における外気に接する床の面積 全ての住戸で 0.0 (m<sup>2</sup>)  
各階における住戸数 1～3階: 4(戸)

In [35]:
get_A_if_h([162.6, 162.6, 162.6],[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [4,4,4])

[[0.0, 0.0, 0.0, 0.0],
 [40.65, 40.65, 40.65, 40.65],
 [40.65, 40.65, 40.65, 40.65]]

### 8) 界天井の面積

階$i$の各住戸における界天井の面積$A_{ic,h,i}$ (m<sup>2</sup>) は、次式により算出する。

$$
A_{ic,h,i} = \left\{
    \begin{array}{ll}
        0 & (i=N)\\
        \frac{ A_{f,h,total,i} }{ n_i } - A_{r,h,i} & ( i < N )
    \end{array}
\right.
$$

$N$: 建物の階数(階)  
$A_{ic,h,i}$: 階$i$の各住戸における界天井の面積 (m<sup>2</sup>)  
$A_{f,h,total,i}$: 階$i$の専有部(住戸部)の床面積の合計 (m<sup>2</sup>)  
$A_{r,h,i}$: 階$i$の各住戸における外気に接する屋根の面積 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数(戸)

In [36]:
def get_A_ic_h(A_f_h_total, A_r_h, n):
    def f(A_f_h_total_i, A_r_h_i, n_i):
        return [ x - y for (x,y) in zip( [A_f_h_total_i / n_i] * n_i, A_r_h_i )]
    return [
        [0.0] * n_i if i == len(A_f_h_total) - 1 else f(A_f_h_total_i, A_r_h_i, n_i)
        for ( i, (A_f_h_total_i, A_r_h_i, n_i) ) in enumerate(zip(A_f_h_total, A_r_h, n))
    ]

#### 計算例

各階の専有部(住戸部)の床面積の合計 1階: 162.6, 2階: 162.6, 3階: 162.6 (m<sup>2</sup>)  
各階の各住戸における外気に接する屋根の面積 1～2階: 0.0, 3階: 40.7 (m<sup>2</sup>)  
各階における住戸数 1～3階: 4(戸)

In [37]:
get_A_ic_h([162.6, 162.6, 162.6],[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [40.7, 40.7, 40.7, 40.7]], [4,4,4])

[[40.65, 40.65, 40.65, 40.65],
 [40.65, 40.65, 40.65, 40.65],
 [0.0, 0.0, 0.0, 0.0]]

### 9) 床下に面する床面積

階$i$の各住戸における床下に面する床面積$A_{csf,h,i}$ (m<sup>2</sup>) は、次式により算出する。

$$
A_{csf,h,i} = \left\{
    \begin{array}{ll}
        \frac{ A_{f,h,total,i} }{ n_i } & (i=1)\\
        0 & ( i \ge 2 )
    \end{array}
\right.
$$

$A_{csf,h,i}$: 階$i$の各住戸における床下に面する床面積 (m<sup>2</sup>)  
$A_{f,h,total,i}$: 階$i$の専有部(住戸部)の床面積の合計 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数(戸)

In [38]:
def get_A_csf_h(A_f_h_total, n):
    return [ [ A_f_h_total_i / n_i ] * n_i if i == 0 else [0.0] * n_i
             for ( i, (A_f_h_total_i, n_i) ) in enumerate(zip(A_f_h_total, n)) ]

#### 計算例

各階の専有部(住戸部)の床面積の合計 1階: 162.6, 2階: 162.6, 3階: 162.6 (m<sup>2</sup>)  
各階における住戸数 1～3階: 4(戸)

In [39]:
get_A_csf_h([162.6, 162.6, 162.6],[4,4,4])

[[40.65, 40.65, 40.65, 40.65], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]

### 3. 各住戸の熱橋長さ

各階における各住戸の構造熱橋部の長さ

| 種類<br>(記号) | 境界の数<br>(屋外/屋内/床下) | 熱橋の方向 | 主な発生場所 |
| --- | --- | --- | --- |
| $ \psi_{1v} $ | 3 / 1 / - | 鉛直 | 外壁同士の接合部 |
| $ \psi_{1h,r} $ | 3 / 1 / - | 水平 | 外壁と屋根の接合部 |
| $ \psi_{1h,f} $ | 3 / 1 / - | 水平 | 外壁と外気床の接合部 |
| $ \psi_{2v} $ | 2 / 2 / - | 鉛直 | 外壁と間仕切壁の接合部（住戸長辺≧20mの場合は追加） |
| $ \psi_{2h,r} $ | 2 / 2 / - | 水平 | 屋根と界壁の接合部（住戸長辺≧20mの場合は追加） |
| $ \psi_{2h,f1} $ | 2 / 2 / - | 水平 | 外壁と界床の接合部 |
| $ \psi_{2h,f2} $ | 2 / 2 / - | 水平 | 外気床と界壁の接合部 |
| $ \psi_{3v} $ | 1 / 3 / - | 鉛直 | 外壁同士の接合部 |
| $ \psi_{3h,r} $ | 1 / 3 / - | 水平 | 外壁と中間階屋根の接合部 |
| $ \psi_{3h,f} $ | 1 / 3 / - | 水平 | 外壁と外気床の接合部 |
| $ \psi_{4h,uf} $ | 2 / 1 / 1 | 水平 | 外壁と最下階床の接合部 |
| $ \psi_{5h,uf} $ | - / 2 / 2 | 水平 | 界壁と最下階床の接合部 |


![HeatBridge](HeatBridgeType.png)

In [40]:
def general(d):
    
    d_general = d['General']
    d_private = d['PrivateArea']
    d_public  = d['PublicArea']
    
    H = get_H(d_general['Height'], d_general['Levels_Number'])
    print('階高: ' + str(H))
        
    A_f_h_total = get_A_f_h_total(d_general['FloorArea'], d_public['FloorArea'])
    print('専有部の床面積の合計: ' + str(A_f_h_total))
    
    l_1_total = get_l_1_total(d_private['PeriLengthOutside'], A_f_h_total)
    l_2_total = get_l_2_total(d_private['PeriLengthOutside'], A_f_h_total)

    print('専有部の全体の長辺の長さ: ' + str(l_1_total))
    print('専有部の全体の短辺の長さ: ' + str(l_2_total))

    A_r_h_total = get_A_r_h_total(d_private['RoofArea'], A_f_h_total)
    print('専有部の屋根面積の合計: ' + str(A_r_h_total))

    A_of_h_total = get_A_of_h_total(d_private['OutsideFloorArea'])
    print('専有部の外気床面積の合計: ' + str(A_of_h_total))
    
    A_win_h_total = get_A_win_h_total(d_private['WindowArea'], d_general['HouseNumber'])
    print('窓面積の合計: ' + str(A_win_h_total))
    
    A_r_h = get_A_r_h(A_r_h_total, d_general['HouseNumber'])
    print('各住戸の屋根面積: ' + str(A_r_h))
    
    A_win_h = get_A_win_h(A_win_h_total,d_general['HouseNumber'])
    print('各住戸の窓面積: ' + str(A_win_h))
    
    A_door_h = get_A_door_h(d_general['HouseNumber'])
    print('各住戸のドアの面積: ' +  str(A_door_h))
    
    A_ow_h = get_A_ow_h( l_1_total, l_2_total, H, d_general['HouseNumber'], A_win_h, A_door_h )
    print('各住戸の外壁の面積: ' + str(A_ow_h))
    
    A_of_h = get_A_of_h( A_of_h_total, d_general['HouseNumber'] )
    print('各住戸の外気床の面積: ' + str(A_of_h))
    
    A_iw_h = get_A_iw_h(l_2_total, H, d_general['HouseNumber'])
    print('各住戸の界壁の面積: ' + str(A_iw_h))
    
    A_if_h = get_A_if_h(A_f_h_total, A_of_h, d_general['HouseNumber'])
    print('各住戸の界床の面積: ' + str(A_if_h))

    A_ic_h = get_A_ic_h(A_f_h_total, A_r_h, d_general['HouseNumber'])
    print('各住戸の界天井の面積: ' + str(A_ic_h))
            
    A_csf_h = get_A_csf_h(A_f_h_total, d_general['HouseNumber'])
    print('各住戸の床下に面する床の面積: ' + str(A_csf_h))
    
    return 0


In [41]:
general(d)

階高: [3.033333333333333, 3.033333333333333, 3.033333333333333]
専有部の床面積の合計: [162.64000000000001, 162.64, 162.64]
専有部の全体の長辺の長さ: [27.32876045198881, 27.328760451988813, 27.328760451988813]
専有部の全体の短辺の長さ: [5.951239548011191, 5.9512395480111895, 5.9512395480111895]
専有部の屋根面積の合計: [0.0, 0.0, 162.64]
専有部の外気床面積の合計: [0.0, 0.0, 0.0]
窓面積の合計: [Direction(D0=18.336768, D90=0.9144, D180=10.314432, D270=0.9144), Direction(D0=18.433024, D90=0.9192, D180=10.368576, D270=0.9192), Direction(D0=18.433024, D90=0.9192, D180=10.368576, D270=0.9192)]
各住戸の屋根面積: [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [40.66, 40.66, 40.66, 40.66]]
各住戸の窓面積: [[Direction(D0=4.584192, D90=0.9144, D180=2.578608, D270=0), Direction(D0=4.584192, D90=0, D180=2.578608, D270=0), Direction(D0=4.584192, D90=0, D180=2.578608, D270=0), Direction(D0=4.584192, D90=0, D180=2.578608, D270=0.9144)], [Direction(D0=4.608256, D90=0.9192, D180=2.592144, D270=0), Direction(D0=4.608256, D90=0, D180=2.592144, D270=0), Direction(D0=4.608256, D90=0, D180=

0